In [2]:
import importlib

import ipynb.fs.full.preprocessing0
import ipynb.fs.full.preprocessing05
#import ipynb.fs.full.prepossessing075_akistage
import preprocessing1
#import ipynb.fs.full.preprocessing2_BT
import preprocessing2_BT

import ipynb.fs.full.preprocessing25_BTcorr
import ipynb.fs.full.preprocessing3_smote
#import ipynb.fs.full.preprocessing4
import preprocessing4

#import ipynb.fs.full.runxgboost
import runxgboost

#import ipynb.fs.full.postprocessing1_SHAP

import postprocessing1_SHAP

import ipynb.fs.full.postprocessing3_collect

from ipynb.fs.full.slackbot import ping_slack
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from os.path import exists
import logging
import time

import scipy
import dataframe_image

In [3]:
importlib.reload(ipynb.fs.full.preprocessing0)
importlib.reload(ipynb.fs.full.preprocessing05)
#importlib.reload(ipynb.fs.full.prepossessing075_akistage)
importlib.reload(preprocessing1)
importlib.reload(preprocessing2_BT)
importlib.reload(ipynb.fs.full.preprocessing25_BTcorr)
importlib.reload(ipynb.fs.full.preprocessing3_smote)
# #importlib.reload(ipynb.fs.full.preprocessing4)
importlib.reload(preprocessing4)
importlib.reload(runxgboost)
importlib.reload(postprocessing1_SHAP)
importlib.reload(ipynb.fs.full.postprocessing3_collect)

#sites = ['MCRI', 'IUR', 'MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UPITT', 'UTHSCSA', 'KUMC', 'UTSW']
sites = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']

stg = 'stg01'
year = 3000

In [49]:
def cal_demo_table(level="encounter"):

    sumeries = list()
    sumerisc = list()

    for site in sites:
        summary = dict()
        summaryc = dict()

        print(site)
        demo = pd.read_parquet('/home/hoyinchan/blue/Data/data2022/'+site+'/p0_demo_'+site+'.parquet')
        demo['PATID'] = demo['PATID'].astype(str)
        demo['ENCOUNTERID'] = demo['ENCOUNTERID'].astype(str)    
        demo['AGE'] = demo['AGE'].astype(float)

        # Remove duplicates death source
        demo = demo.loc[:, ~demo.columns.str.contains("DEATH", case=False)].drop_duplicates()

        final_model_data = pd.read_pickle('/home/hoyinchan/blue/Data/data2022/'+site+'/bt3pos_'+site+'_'+'stg01'+'_3000.pkl')
        final_model_data['PATID'] = final_model_data['PATID'].astype(str)
        final_model_data['ENCOUNTERID'] = final_model_data['ENCOUNTERID'].astype(str)

        onset = pd.read_parquet('/home/hoyinchan/blue/Data/data2022/'+site+'/p0_onset_'+site+'.parquet')
        onset['PATID'] = onset['PATID'].astype(str)
        onset['ENCOUNTERID'] = onset['ENCOUNTERID'].astype(str)

        onset = onset.merge(final_model_data[['PATID', 'ENCOUNTERID']], on = ['PATID', 'ENCOUNTERID'], how = 'inner')

        onset['AKI3'] = onset['AKI_STAGE']==3
        onset['AKI2'] = onset['AKI_STAGE']==2
        onset['AnyAKI'] = onset['AKI_STAGE']!=0
        onset['Non-AKI'] = onset['AKI_STAGE']==0

        onset2 = onset.merge(demo, on = ['PATID', 'ENCOUNTERID'], how='left')

        #First Encounter
        if level != "encounter":
            onset2 = onset2.sort_values('ADMIT_DATE').groupby(['PATID']).first().reset_index()

        racial_categories = {
            "01": "Native American",
            "02": "Asian",
            "03": "Black",
            "04": "Not specified",
            "05": "White",
            "OT": "Not specified",
            "NS": "Not specified"
        }

        hispanic_categories = {
            "Y" : "Yes",
            "N": "No",
            "NI": "Unknown",
            "UN" : "Unknown"
        }


        sex_categories = {
            "M" : "Male",
            "F": "Female",
            "NI": None,
            "UN" : None,
            "A" : None
        }    

        df = onset2
        df['RACE'] = df['RACE'].map(racial_categories)
        df['HISPANIC'] = df['HISPANIC'].map(hispanic_categories)
        df['SEX'] = df['SEX'].map(sex_categories)

        # Total count
        total_count = len(df)
        summary['N'] = "{:,}".format(total_count)
        summaryc['N'] = total_count

        # Categorize Age
        age_bins = [0, 25, 35, 45, 55, 65, float('inf')]
        age_labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '\u2265 66']
        df['Age_Category'] = pd.cut(df['AGE'], bins=age_bins, labels=age_labels, right=False)

        df = df[['Non-AKI', 'AnyAKI', 'AKI2', 'AKI3', 'Age_Category', 'SEX', 'HISPANIC', 'RACE']]
        df.columns = ['Non-AKI', 'Any AKI', 'AKI2', 'AKI3', 'Age', 'Sex', 'Hispanic', 'Race']
        # Counting and reshaping
        for column in df.columns:
            if 'AKI' in column:
                count = df[column].sum()
                percentage = (count / total_count) * 100
                summary[f'AKI:{column}'] = "{:,}".format(count)+f" ({percentage:.1f}%)"
                summaryc[column] = count
            else:
                counts = df[column].value_counts()
                for value in counts.index:
                    count = counts[value]
                    percentage = (count / total_count) * 100
                    summary[f'{column}:{value}'] = "{:,}".format(count)+f" ({percentage:.1f}%)"
                    summaryc[f'{column}:{value}'] = count

        # Creating a single-row DataFrame
        summary_df = pd.DataFrame([summary])
        summary_df.index = [site]
        sumeries.append(summary_df)

        # Creating a single-row DataFrame
        summary_dfx = pd.DataFrame([summaryc])
        summary_dfx.index = [site]
        sumerisc.append(summary_dfx)    

    sumall = pd.concat(sumeries)
    sumallx = pd.concat(sumerisc)

    return sumall, sumallx

In [50]:
sumall_enc, sumallx_enc = cal_demo_table(level="encounter")

KUMC
UTSW
MCW
UofU
UIOWA
UMHC
UPITT
UTHSCSA
UNMC


In [51]:
sumall_p, sumallx_p = cal_demo_table(level="paitient")

KUMC
UTSW
MCW
UofU
UIOWA
UMHC
UPITT
UTHSCSA
UNMC


In [52]:
sumall = sumall_p

In [53]:
sumall[['AKI:Non-AKI', 'AKI:Any AKI', 'AKI:AKI2', 'AKI:AKI3']] = sumall_enc[['AKI:Non-AKI', 'AKI:Any AKI', 'AKI:AKI2', 'AKI:AKI3']]

In [54]:
sumall = sumall[['N', 
                  'AKI:Non-AKI', 'AKI:Any AKI', 'AKI:AKI2', 'AKI:AKI3', 
                  'Age:18-25', 'Age:26-35', 'Age:36-45', 'Age:46-55', 'Age:56-65', 'Age:≥ 66',
                  'Sex:Male', 'Sex:Female', 
                  'Hispanic:Yes', 'Hispanic:No', 'Hispanic:Unknown', 
                  'Race:Asian', 'Race:Black', 'Race:Native American', 'Race:White', 'Race:Not specified']]

sumall = sumall.drop(['AKI:AKI2', 'AKI:AKI3'],axis=1)

In [55]:
site_key = {3:'MCW', 4:'UIOWA', 5:'UMHC', 6:'UNMC', 9:'UofU', 8:'UTHSCSA', 2:'KUMC', 1:'UTSW', 7:'UPITT'}
site_keyr = {v: k for k, v in site_key.items()}
sumallt = sumall.T
sumallt = sumallt.fillna('0 (0.0%)')
sumallt = sumallt.rename(columns=site_keyr)
sumallt = sumallt.reindex(sorted(sumallt.columns), axis=1)
sumallt.to_csv('data_description.csv')

# Splitting the index and creating a MultiIndex for hierarchical display
new_index = pd.MultiIndex.from_tuples([tuple(idx.split(':')) if ':' in idx else (idx, '') for idx in sumallt.index])
sumallt.index = new_index

# Expand the DataFrame to insert empty rows for each new category
new_rows = []
prev_cat = None
for idx, row in sumallt.iterrows():
    cat, subcat = idx
    if cat != prev_cat:
        new_rows.append(((cat, ''), pd.Series([None]*len(sumallt.columns), index=sumallt.columns)))
        prev_cat = cat
    new_rows.append((idx, row))

expanded_df = pd.DataFrame(dict(new_rows)).T

def remove_cat(val):
    if val[1] != '':
        val = ('', val[1])
    return val

expanded_df.index = pd.MultiIndex.from_tuples([remove_cat(x) for x in expanded_df.index])

# Function to replace None with empty string
def format_none(val):
    return '' if val is None else val

# Styling the DataFrame with indentation for subcategories
styled_df = expanded_df.style.apply(
    lambda x: ['padding-left: 20px' if x.name[1] else '' for _ in x], axis=1
                ).set_caption('Table 1 Demographic characteristics at different health systems.')\
                .format(format_none)\
                .set_table_styles([
                        # Aligning all cells to the left
                        {'selector': 'td', 'props': [('text-align', 'left')]},    
                        {'selector': 'th', 'props': [('text-align', 'left')]},  # Aligning index to the left

                        # Optional: Adding a border under the column headers for consistency
                        {'selector': 'thead th', 'props': [('border-bottom', '1px solid black')]},
                        
                        # Style for the caption
                        {'selector': 'caption', 
                         'props': [('color', 'black'), 
                                   ('background-color', 'yellow'), 
                                   ('font-size', '16px'),
                                   ('text-align', 'left'),
                                   ('font-weight', 'bold')]},                    

                ])

dataframe_image.export(styled_df,"styled_table.png", table_conversion="playwright")
styled_df

In [56]:
sumall

,N,AKI:Non-AKI,AKI:Any AKI,Age:18-25,Age:26-35,Age:36-45,Age:46-55,Age:56-65,Age:≥ 66,Sex:Male,Sex:Female,Hispanic:Yes,Hispanic:No,Hispanic:Unknown,Race:Asian,Race:Black,Race:Native American,Race:White,Race:Not specified
KUMC,"106,479","182,642 (86.5%)","28,500 (13.5%)","5,155 (4.8%)","9,366 (8.8%)","11,061 (10.4%)","16,774 (15.8%)","24,301 (22.8%)","39,822 (37.4%)","54,350 (51.0%)","52,127 (49.0%)","6,268 (5.9%)","99,063 (93.0%)","1,148 (1.1%)","1,060 (1.0%)","14,030 (13.2%)",421 (0.4%),"80,700 (75.8%)","9,354 (8.8%)"
UTSW,"57,246","92,583 (85.3%)","15,894 (14.7%)","1,988 (3.5%)","4,149 (7.2%)","5,521 (9.6%)","8,597 (15.0%)","12,764 (22.3%)","24,227 (42.3%)","27,623 (48.3%)","29,622 (51.7%)","6,990 (12.2%)","46,551 (81.3%)","3,705 (6.5%)","1,301 (2.3%)","10,224 (17.9%)",131 (0.2%),"37,361 (65.3%)","3,134 (5.5%)"
MCW,"69,399","107,585 (86.7%)","16,508 (13.3%)","3,097 (4.5%)","5,672 (8.2%)","6,269 (9.0%)","10,638 (15.3%)","15,946 (23.0%)","27,777 (40.0%)","35,526 (51.2%)","33,873 (48.8%)","2,348 (3.4%)","66,762 (96.2%)",240 (0.3%),862 (1.2%),"13,984 (20.2%)",248 (0.4%),"51,547 (74.3%)","1,426 (2.1%)"
UofU,"69,752","100,473 (90.2%)","10,930 (9.8%)","3,936 (5.6%)","7,513 (10.8%)","8,386 (12.0%)","10,529 (15.1%)","14,905 (21.4%)","24,483 (35.1%)","36,603 (52.5%)","33,148 (47.5%)","5,729 (8.2%)","60,823 (87.2%)","2,564 (3.7%)",986 (1.4%),"1,183 (1.7%)","1,383 (2.0%)","58,387 (83.7%)","6,406 (9.2%)"
UIOWA,"82,256","115,904 (86.6%)","17,875 (13.4%)","3,870 (4.7%)","6,674 (8.1%)","8,050 (9.8%)","13,710 (16.7%)","18,803 (22.9%)","31,149 (37.9%)","42,360 (51.5%)","39,895 (48.5%)","2,137 (2.6%)","75,888 (92.3%)","3,469 (4.2%)",811 (1.0%),"3,815 (4.6%)",260 (0.3%),"73,950 (89.9%)",88 (0.1%)
UMHC,"47,799","64,575 (85.7%)","10,775 (14.3%)","2,930 (6.1%)","4,460 (9.3%)","5,041 (10.5%)","7,510 (15.7%)","10,042 (21.0%)","17,816 (37.3%)","24,189 (50.6%)","23,610 (49.4%)",805 (1.7%),"44,744 (93.6%)",401 (0.8%),230 (0.5%),"3,620 (7.6%)",93 (0.2%),"42,531 (89.0%)",598 (1.3%)
UPITT,"223,949","331,690 (86.3%)","52,724 (13.7%)","6,540 (2.9%)","13,195 (5.9%)","15,488 (6.9%)","25,568 (11.4%)","44,014 (19.7%)","119,144 (53.2%)","107,425 (48.0%)","116,524 (52.0%)","1,223 (0.5%)","203,093 (90.7%)","14,846 (6.6%)","1,557 (0.7%)","21,286 (9.5%)",396 (0.2%),"192,148 (85.8%)",117 (0.1%)
UTHSCSA,"69,064","87,495 (83.8%)","16,907 (16.2%)","5,639 (8.2%)","9,559 (13.8%)","10,001 (14.5%)","13,756 (19.9%)","15,085 (21.8%)","15,024 (21.8%)","36,910 (53.4%)","32,111 (46.5%)","42,812 (62.0%)","25,580 (37.0%)",591 (0.9%),935 (1.4%),"4,354 (6.3%)",74 (0.1%),"61,433 (89.0%)",67 (0.1%)
UNMC,"59,553","87,438 (86.2%)","14,018 (13.8%)","2,842 (4.8%)","5,136 (8.6%)","5,811 (9.8%)","8,699 (14.6%)","12,902 (21.7%)","24,163 (40.6%)","30,493 (51.2%)","29,058 (48.8%)","3,333 (5.6%)","55,946 (93.9%)",225 (0.4%),567 (1.0%),"5,373 (9.0%)",496 (0.8%),"49,050 (82.4%)","2,812 (4.7%)"


In [57]:
fitdata = pd.read_csv("MetaRegression/plotmp.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'MetaRegression/plotmp.csv'

In [ ]:
def extra_translate_omop(label):
    # first_dixt = {x:plotshapsn.translate_omop(x) for x in plotshapsn.shapdf['Feature'].unique()}
    # Then ask ChatGPT to extract "extract the medical procedure, medication or Lab measurement from..."
    data_dict = {
        'AGE': 'AGE',
        'PX:09:96.72': 'Invasive Mechanical Ventilation',
        'PX:CH:J1940': 'Furosemide Injection',
        'SYSTOLIC': 'SYSTOLIC',
        'DIASTOLIC': 'DIASTOLIC',
        'ORIGINAL_BMI': 'BMI',
        'LAB::17861-6(mg/dL)': 'Calcium',
        'LAB::1975-2(mg/dL)': 'Total Bilirubin',
        'LAB::2075-0(mmol/L)': 'Chloride',
        'LAB::2160-0(mg/dL)': 'Creatinine',
        'LAB::2345-7(mg/dL)': 'Glucose',
        'LAB::2777-1(mg/dL)': 'Phosphate',
        'LAB::2823-3(mmol/L)': 'Potassium',
        'LAB::2951-2(mmol/L)': 'Sodium',
        'LAB::3094-0(mg/dL)': 'Urea Nitrogen',
        'LAB::33037-3(mmol/L)': 'Anion Gap',
        'LAB::4092-3(ug/mL)': 'Vancomycin',
        'LAB::4544-3(%)': 'Hematocrit',
        'LAB::6690-2(10*3/uL)': 'Leukocyte',
        'LAB::718-7(g/dL)': 'Hemoglobin',
        'LAB::777-3(10*3/uL)': 'Platelet',
        'LAB::788-0(%)': 'RDW',
        'LAB::2028-9(mmol/L)': 'Carbon dioxide'
    }
    return data_dict.get(label, label)

In [ ]:
styled_df

In [ ]:
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

df = expanded_df
outputname = 'demo'
df.index.names = ['First Level', 'Second Level']
index = df.index
df.columns = [str(i) for i in df.columns]
df = df.fillna('')

In [ ]:
# Create a new Document
doc = Document()
# Adjust for multi-index (+1 for each level of the multi-index)
table = doc.add_table(rows=df.shape[0]+1, cols=len(df.columns) + len(index.levels))

# Insert the column names for the index levels and then for the data columns
hdr_cells = table.rows[0].cells
for i, name in enumerate(index.names):
    run = hdr_cells[i].paragraphs[0].add_run(name)
    run.bold = True
    hdr_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

for j, column in enumerate(df.columns, start=len(index.names)):
    run = hdr_cells[j].paragraphs[0].add_run(column)
    run.bold = True
    hdr_cells[j].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

# Insert the row data
for row_index, row in df.iterrows():
    row_cells = table.add_row().cells
    # Write the multi-index values and make them bold
    for i, index_part in enumerate(row_index):
        run = row_cells[i].paragraphs[0].add_run(str(index_part))
        run.bold = True
        row_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
    # Write the data columns with appropriate formatting
    for j, (column_name, value) in enumerate(row.items(), start=len(index.names)):
        text = f"{value:.6f}" if isinstance(value, float) else str(value)
        run = row_cells[j].paragraphs[0].add_run(text)
        row_cells[j].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        
doc.save(outputname+'.docx')  